In [1]:
import pandas as pd
import pyranges as pr
mt = pd.read_table('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/filtered_metadata_ENCODE_in_unibind.tsv', sep='\t')

In [2]:
mt['sample_name'] = mt['sample_name'] + '_' + mt['Experiment.accession']

In [3]:
len(set(mt['sample_name']))

309

In [6]:
import os
path_to_region_sets = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/'
region_sets = {mt['sample_name'].tolist()[i]:pr.PyRanges(pd.read_table(os.path.join(path_to_region_sets, mt['File.accession'].tolist()[i]+'.bed.gz'), header=None).iloc[:,0:3].rename(columns={0: "Chromosome", 1: "Start", 2:'End'})) for i in range(mt.shape[0])}

In [ ]:
import os
path_to_region_sets = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/'
for i in range(mt.shape[0]):
    x = pd.read_table(os.path.join(path_to_region_sets, mt['File.accession'].tolist()[i]+'.bed.gz'))
    x.to_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/renamed_files/'+mt['sample_name'].tolist()[i]+'.bed', sep='\t', index=False)                  

In [7]:
# Save
import pickle
with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'wb') as f:
    pickle.dump(region_sets, f )

In [19]:
import os
path_to_region_sets = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/'
region_sets = {mt['sample_name'].tolist()[i]:pr.PyRanges(pd.read_table(os.path.join(path_to_region_sets, mt['File.accession'].tolist()[i]+'.bed.gz'), header=None).iloc[0:10000,0:3].rename(columns={0: "Chromosome", 1: "Start", 2:'End'})) for i in range(mt.shape[0])}

In [20]:
# Save
import pickle
with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets_top10K.pkl', 'wb') as f:
    pickle.dump(region_sets, f )

### Homer

In [ ]:
# r06i01n12
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

In [6]:
import pandas as pd
d = {key: len(region_sets[key]) for key in region_sets.keys()}
pd.DataFrame.from_dict(d, orient='index').to_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/peaks_per_tracks.tsv', sep='\t')

In [22]:
[len(region_sets[x]) for x in region_sets.keys()]

[67469,
 108366,
 179494,
 61908,
 125643,
 53098,
 103955,
 70443,
 17321,
 91100,
 90352,
 70978,
 106863,
 22580,
 7287,
 81028,
 54541,
 29740,
 87682,
 55623,
 33898,
 91620,
 82664,
 53884,
 53704,
 53892,
 115989,
 19758,
 93381,
 102856,
 54152,
 56289,
 47087,
 77848,
 62923,
 71639,
 36261,
 66033,
 85064,
 126789,
 55092,
 115394,
 71183,
 112272,
 143635,
 107386,
 142238,
 48068,
 61093,
 57233,
 113436,
 27112,
 186214,
 105499,
 67441,
 48347,
 852,
 31747,
 86395,
 44994,
 29647,
 126206,
 14936,
 129299,
 45496,
 32108,
 8505,
 71613,
 34847,
 61778,
 18496,
 46183,
 55729,
 41879,
 25478,
 127735,
 97296,
 30295,
 126732,
 50630,
 90942,
 117635,
 95382,
 50610,
 25435,
 45433,
 31703,
 53784,
 76702,
 82458,
 24631,
 7366,
 99139,
 8934,
 69198,
 138533,
 63740,
 146704,
 121651,
 110500,
 113735,
 65731,
 87738,
 132647,
 69816,
 111666,
 94572,
 30958,
 49732,
 94176,
 61212,
 116286,
 24094,
 130774,
 27297,
 57879,
 110589,
 146776,
 78209,
 30214,
 162988,
 1060

In [ ]:
# Load homer functions
from pycistarget.motif_enrichment_homer import *

In [ ]:
# Set correct path to run HOMER
import os
os.putenv('HOMER_HOME','/data/leuven/software/biomed/haswell_centos7/2018a/software/HOMER/4.10.4-foss-2018a')
os.environ["PATH"] += os.pathsep + '/data/leuven/software/biomed/haswell_centos7/2018a/software/HOMER/4.10.4-foss-2018a/bin:'
homer_path='/data/leuven/software/biomed/haswell_centos7/2018a/software/HOMER/4.10.4-foss-2018a/bin/'
# Choose the output directory for the results
outdir='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/'
# Select your genome
genome='hg38'
# Set correct path to MEME for de novo motif annotation
os.putenv('MEME_HOME','/data/leuven/software/biomed/haswell_centos7/2018a/software/MEME/5.1.1-foss-2018a-Perl-5.28.1-Python-3.6.4')
os.environ["PATH"] += os.pathsep + '/data/leuven/software/biomed/haswell_centos7/2018a/software/MEME/5.1.1-foss-2018a-Perl-5.28.1-Python-3.6.4/bin:'
meme_collection_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/pycistarget/tutorial/ctx_v9_motif_collection.meme'
meme_path='/data/leuven/software/biomed/haswell_centos7/2018a/software/MEME/5.1.1-foss-2018a-Perl-5.28.1-Python-3.6.4/bin/'
# Run
homer_dict=run_homer(homer_path,
                     region_sets,
                     outdir,
                     genome,
                     size='given',
                     mask=True,
                     denovo=False,
                     length='8,10,12',
                     n_cpu=8,
                     meme_path = meme_path,
                     meme_collection_path = meme_collection_path,
                     cistrome_annotation = ['Direct_annot', 'Motif_similarity_annot', 'Orthology_annot', 'Motif_similarity_and_Orthology_annot'],
                     _temp_dir='/scratch/leuven/313/vsc31305/ray_spill')

In [ ]:
# Save pkl
import pickle
with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/homer_dict.pkl', 'wb') as f:
  pickle.dump(homer_dict, f)

In [ ]:
import pandas as pd
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/tsv/'
for key in homer_dict.keys():
    homer_dict[key].known_motifs.to_csv(dire+key+'.tsv', sep='\t')

In [ ]:
import pandas as pd
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/tsv/'
for key in homer_dict.keys():
    homer_dict[key].known_motifs.to_csv(dire+key+'.tsv', sep='\t')

In [ ]:
import pandas as pd
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/cistromes_hits/'
for key in homer_dict.keys():
    tf = key.split('_')[1]
    dict_keys = homer_dict[key].known_cistromes.keys()
    name = [name for name in dict_keys if tf+' (' in name]
    if len(name) == 1:
        save_name = name[0].split(' (')[0]
        pd.DataFrame(homer_dict[key].known_cistromes[name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
    name = [name for name in dict_keys if tf+'_extended (' in name]
    if len(name) == 1:
        save_name = name[0].split(' (')[0]
        pd.DataFrame(homer_dict[key].known_cistromes[name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [35]:
def region_names_to_coordinates(region_names):
    chrom = pd.DataFrame([i.split(':', 1)[0]
                         for i in region_names if ':' in i])
    coor = [i.split(':', 1)[1] for i in region_names if ':' in i]
    start = pd.DataFrame([int(i.split('-', 1)[0]) for i in coor])
    end = pd.DataFrame([int(i.split('-', 1)[1]) for i in coor])
    regiondf = pd.concat([chrom, start, end], axis=1, sort=False)
    regiondf.index = [i for i in region_names if ':' in i]
    regiondf.columns = ['Chromosome', 'Start', 'End']
    return (regiondf)
def coord_to_region_names(coord):
    """
    PyRanges to region names
    """
    if isinstance(coord, pr.PyRanges):
        coord = coord.as_df()
        return list(coord['Chromosome'].astype(str) + ':' + coord['Start'].astype(str) + '-' + coord['End'].astype(str))

In [28]:
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/cistromes_hits/'
import pyranges as pr
import os
import pandas as pd
files = [dire+x for x in os.listdir(dire)]
x = files[0]
tracks = {os.path.basename(x):pr.PyRanges(region_names_to_coordinates(pd.read_csv(x, header=None)[0].tolist())) for x in files}

In [29]:
consensus_peaks = pr.read_bed('/staging/leuven/stg_00002/lcb/icistarget/data/regions/homo_sapiens/hg38/SCREEN3/V2.hg38-rDHS-Unfiltered.blacklisted.max_masked_perc_0.99.bed')

In [32]:
tracks_to_set = {x:consensus_peaks.join(tracks[x]).df[['Chromosome', 'Start', 'End']].drop_duplicates(subset=['Chromosome', 'Start', 'End']) for x in tracks.keys()}

In [41]:
save_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/Homer/cistromes_hits_SCREEN/'
for x in tracks_to_set.keys():
    pd.DataFrame(coord_to_region_names(pr.PyRanges(tracks_to_set[x]))).to_csv(save_path+x, sep='\t', header=False, index=False)

In [38]:
coord_to_region_names(tracks_to_set['K562_TAL1_ENCSR106FRG_TAL1_extended.tsv'])

### V10 - Clustered - cistarget

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [1]:
import scenicplus
scenicplus.__version__
from scenicplus.TF_to_gene import _inject_TF_as_its_own_target

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

In [ ]:
# cisTarget - All
# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *
for i in range(len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/hg38_SCREEN_cluster_db/cluster_SCREEN.regions_vs_motifs.rankings.feather'  
    rg = {key: region_sets[key]}
    ctx_db = cisTargetDatabase(db, rg)    
    # Run (r06i00n12)
    cistarget_dict = run_cistarget(ctx_db = ctx_db,
                                   region_sets = rg,
                                   specie = 'homo_sapiens',
                                   auc_threshold = 0.005,
                                   nes_threshold = 3.0,
                                   rank_threshold = 0.05,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/cbravo/cluster_motif_collection_V10_no_desso_no_factorbook/snapshots/motifs-v10-nr.more_orthology.hgnc-mm0.00001-o0.0_clust.tsv',
                                   n_cpu = 1,
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10nr_clust')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/output_dicts/' + key + '_cisTarget_dict.pkl', 'wb') as f:
      pickle.dump(cistarget_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/html/'+ key + '.html'
    cistarget_dict[key].motif_enrichment.to_html(open(out_file, 'w'), escape=False, col_space=80)

### V10 - Archetypes - cistarget

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

In [ ]:
# cisTarget - All
# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *
for i in range(len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/hg38_SCREEN_cluster_db_archetype/cluster_SCREEN_archetype.regions_vs_motifs.rankings.feather'  
    rg = {key: region_sets[key]}
    ctx_db = cisTargetDatabase(db, rg)    
    # Run (r06i00n12)
    cistarget_dict = run_cistarget(ctx_db = ctx_db,
                                   region_sets = rg,
                                   specie = 'homo_sapiens',
                                   auc_threshold = 0.005,
                                   nes_threshold = 3.0,
                                   rank_threshold = 0.05,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/cbravo/cluster_motif_collection_V10_no_desso_no_factorbook/snapshots/motifs-v10-nr.more_orthology.hgnc-mm0.00001-o0.0_clust.tsv',
                                   n_cpu = 1,
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10nr_clust')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/output_dicts/' + key + '_cisTarget_dict.pkl', 'wb') as f:
      pickle.dump(cistarget_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/html/'+ key + '.html'
    cistarget_dict[key].motif_enrichment.to_html(open(out_file, 'w'), escape=False, col_space=80)

### V10 - Clustered - DEM

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

In [ ]:
# Prepare annotation
import pybiomart as pbm
import pandas as pd
import pyranges as pr
biomart_host = 'http://www.ensembl.org'
promoter_space = 500
name = 'hsapiens_gene_ensembl'
dem_log2fc_thr = 0.5,
dem_motif_hit_thr = 3.0,
dem_max_bg_regions = 500,
dataset = pbm.Dataset(name=name,  host=biomart_host)
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot['Chromosome'] = annot['Chromosome'].astype('str')
filterf = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
annot = annot[~filterf]
annot['Chromosome'] = annot['Chromosome'].str.replace(r'(\b\S)', r'chr\1')
annot = annot[annot.Transcript_type == 'protein_coding']
annot = annot.dropna(subset = ['Chromosome', 'Start'])
# Check if chromosomes have chr
check = region_sets[list(region_sets.keys())[0]]
if not any(['chr' in c for c in check[list(check.keys())[0]].df['Chromosome']]):
    annot.Chromosome = annot.Chromosome.str.replace('chr', '')
annot_dem=annot.copy()
# Define promoter space
annot['End'] = annot['Start'].astype(int)+promoter_space
annot['Start'] = annot['Start'].astype(int)-promoter_space
annot = pr.PyRanges(annot[['Chromosome', 'Start', 'End']])

In [9]:
region_set_list = [region_sets[x].df for x in region_sets.keys()]
all_regions = pr.PyRanges(pd.concat(region_set_list))

In [ ]:
# DEM
# Load DEM functions
from pycistarget.motif_enrichment_dem import *
for i in range(275,len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Create bg
    bg_overlap = all_regions.count_overlaps(region_sets[key])
    bg = bg_overlap[bg_overlap.NumberOverlaps == 0]
    bg = pr.PyRanges(bg.df.sample(frac=0.0005))
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/hg38_SCREEN_cluster_db/cluster_SCREEN.regions_vs_motifs.scores.feather'  
    rg = {key: region_sets[key], 'Background': bg}
    dem_db = DEMDatabase(db, rg)    
    # Run (r06i00n12)
    DEM_dict = DEM(dem_db = dem_db,
                                   region_sets = rg,
                                   contrasts = [[[key],['Background']]],
                                   log2fc_thr = 0.5,
                                   motif_hit_thr = 3.0,
                                   specie = 'homo_sapiens',
                                   max_bg_regions = 5000,
                                   genome_annotation = annot_dem,
                                   promoter_space = promoter_space,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/cbravo/cluster_motif_collection_V10_no_desso_no_factorbook/snapshots/motifs-v10-nr.more_orthology.hgnc-mm0.00001-o0.0_clust.tsv',
                                   n_cpu = 1,
                                   tmp_dir = '/scratch/leuven/313/vsc31305/ray_spill',
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10nr_clust')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/output_dicts/' + key + '_DEM_dict.pkl', 'wb') as f:
      pickle.dump(DEM_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/html/'+ key + '.html'
    DEM_dict.motif_enrichment[key+'_VS_Background'].to_html(open(out_file, 'w'), escape=False, col_space=80)

### V10 - Archetypes - DEM

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

In [ ]:
# Prepare annotation
import pybiomart as pbm
import pandas as pd
import pyranges as pr
biomart_host = 'http://www.ensembl.org'
promoter_space = 500
name = 'hsapiens_gene_ensembl'
dem_log2fc_thr = 0.5,
dem_motif_hit_thr = 3.0,
dem_max_bg_regions = 500,
dataset = pbm.Dataset(name=name,  host=biomart_host)
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot['Chromosome'] = annot['Chromosome'].astype('str')
filterf = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
annot = annot[~filterf]
annot['Chromosome'] = annot['Chromosome'].str.replace(r'(\b\S)', r'chr\1')
annot = annot[annot.Transcript_type == 'protein_coding']
annot = annot.dropna(subset = ['Chromosome', 'Start'])
# Check if chromosomes have chr
check = region_sets[list(region_sets.keys())[0]]
if not any(['chr' in c for c in check[list(check.keys())[0]].df['Chromosome']]):
    annot.Chromosome = annot.Chromosome.str.replace('chr', '')
annot_dem=annot.copy()
# Define promoter space
annot['End'] = annot['Start'].astype(int)+promoter_space
annot['Start'] = annot['Start'].astype(int)-promoter_space
annot = pr.PyRanges(annot[['Chromosome', 'Start', 'End']])

In [9]:
region_set_list = [region_sets[x].df for x in region_sets.keys()]
all_regions = pr.PyRanges(pd.concat(region_set_list))

In [ ]:
# DEM
# Load DEM functions
# Splitting across nodes
from pycistarget.motif_enrichment_dem import *
for i in range(160, 201):
    print(i)
    key = list(region_sets.keys())[i]
    # Create bg
    bg_overlap = all_regions.count_overlaps(region_sets[key])
    bg = bg_overlap[bg_overlap.NumberOverlaps == 0]
    bg = pr.PyRanges(bg.df.sample(frac=0.0005))
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/hg38_SCREEN_cluster_db_archetype/cluster_SCREEN_archetype.regions_vs_motifs.scores.feather'  
    rg = {key: region_sets[key], 'Background': bg}
    dem_db = DEMDatabase(db, rg)    
    # Run (r06i00n12)
    DEM_dict = DEM(dem_db = dem_db,
                                   region_sets = rg,
                                   contrasts = [[[key],['Background']]],
                                   log2fc_thr = 0.5,
                                   motif_hit_thr = 3.0,
                                   specie = 'homo_sapiens',
                                   max_bg_regions = 5000,
                                   genome_annotation = annot_dem,
                                   promoter_space = promoter_space,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/cbravo/cluster_motif_collection_V10_no_desso_no_factorbook/snapshots/motifs-v10-nr.more_orthology.hgnc-mm0.00001-o0.0_clust.tsv',
                                   n_cpu = 1,
                                   tmp_dir = '/scratch/leuven/313/vsc31305/ray_spill',
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10nr_clust')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/output_dicts/' + key + '_DEM_dict.pkl', 'wb') as f:
      pickle.dump(DEM_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/html/'+ key + '.html'
    DEM_dict.motif_enrichment[key+'_VS_Background'].to_html(open(out_file, 'w'), escape=False, col_space=80)

In [ ]:
# DEM
# Load DEM functions
from pycistarget.motif_enrichment_dem import *
for i in range(200,len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Create bg
    bg_overlap = all_regions.count_overlaps(region_sets[key])
    bg = bg_overlap[bg_overlap.NumberOverlaps == 0]
    bg = pr.PyRanges(bg.df.sample(frac=0.0005))
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/hg38_SCREEN_cluster_db_archetype/cluster_SCREEN_archetype.regions_vs_motifs.scores.feather'  
    rg = {key: region_sets[key], 'Background': bg}
    dem_db = DEMDatabase(db, rg)    
    # Run (r06i00n12)
    DEM_dict = DEM(dem_db = dem_db,
                                   region_sets = rg,
                                   contrasts = [[[key],['Background']]],
                                   log2fc_thr = 0.5,
                                   motif_hit_thr = 3.0,
                                   specie = 'homo_sapiens',
                                   max_bg_regions = 5000,
                                   genome_annotation = annot_dem,
                                   promoter_space = promoter_space,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/cbravo/cluster_motif_collection_V10_no_desso_no_factorbook/snapshots/motifs-v10-nr.more_orthology.hgnc-mm0.00001-o0.0_clust.tsv',
                                   n_cpu = 1,
                                   tmp_dir = '/scratch/leuven/313/vsc31305/ray_spill',
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10nr_clust')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/output_dicts/' + key + '_DEM_dict.pkl', 'wb') as f:
      pickle.dump(DEM_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/html/'+ key + '.html'
    DEM_dict.motif_enrichment[key+'_VS_Background'].to_html(open(out_file, 'w'), escape=False, col_space=80)

In [ ]:
# DEM
# Load DEM functions
# Splitting across nodes
from pycistarget.motif_enrichment_dem import *
for i in range(155, 160):
    print(i)
    key = list(region_sets.keys())[i]
    # Create bg
    bg_overlap = all_regions.count_overlaps(region_sets[key])
    bg = bg_overlap[bg_overlap.NumberOverlaps == 0]
    bg = pr.PyRanges(bg.df.sample(frac=0.0005))
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/hg38_SCREEN_cluster_db_archetype/cluster_SCREEN_archetype.regions_vs_motifs.scores.feather'  
    rg = {key: region_sets[key], 'Background': bg}
    dem_db = DEMDatabase(db, rg)    
    # Run (r06i00n12)
    DEM_dict = DEM(dem_db = dem_db,
                                   region_sets = rg,
                                   contrasts = [[[key],['Background']]],
                                   log2fc_thr = 0.5,
                                   motif_hit_thr = 3.0,
                                   specie = 'homo_sapiens',
                                   max_bg_regions = 5000,
                                   genome_annotation = annot_dem,
                                   promoter_space = promoter_space,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/cbravo/cluster_motif_collection_V10_no_desso_no_factorbook/snapshots/motifs-v10-nr.more_orthology.hgnc-mm0.00001-o0.0_clust.tsv',
                                   n_cpu = 1,
                                   tmp_dir = '/scratch/leuven/313/vsc31305/ray_spill',
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10nr_clust')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/output_dicts/' + key + '_DEM_dict.pkl', 'wb') as f:
      pickle.dump(DEM_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/html/'+ key + '.html'
    DEM_dict.motif_enrichment[key+'_VS_Background'].to_html(open(out_file, 'w'), escape=False, col_space=80)

### V10 - Unclustered

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

In [3]:
# Keep only annotated motifs
from pycistarget.utils import *
annot = load_motif_annotations('homo_sapiens',  fname='/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf_project/motif_to_tf_db_data/snapshots/old_20220310/motifs-v10-nr.more_orthology.hgnc-m0.00001-o0.0.tbl')
annot = annot[['Direct_annot', 'Orthology_annot']]
annot = annot.dropna(axis=0, how='all')
motifs = list(set(annot.index))

In [4]:
motifs

24309

In [ ]:
# cisTarget - All
# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *
for i in range(213,len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/icistarget/data/make_rankings/v10/CTX_hg38/CTX_hg38_SCREEN3_1kb_bg_with_mask/CTX_hg38_SCREEN3_1kb_bg_with_mask.regions_vs_motifs.rankings.feather'  
    rg = {key: region_sets[key]}
    ctx_db = cisTargetDatabase(db, rg)   
    ctx_db.db_rankings = ctx_db.db_rankings[ctx_db.db_rankings.index.isin(motifs)]
    # Run (r06i00n12)
    cistarget_dict = run_cistarget(ctx_db = ctx_db,
                                   region_sets = rg,
                                   specie = 'homo_sapiens',
                                   auc_threshold = 0.005,
                                   nes_threshold = 3.0,
                                   rank_threshold = 0.05,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf_project/motif_to_tf_db_data/snapshots/motifs-v10-nr.more_orthology.hgnc-m0.00001-o0.0.tbl',
                                   n_cpu = 1,
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v10')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/output_dicts/' + key + '_cisTarget_dict.pkl', 'wb') as f:
      pickle.dump(cistarget_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/html/'+ key + '.html'
    cistarget_dict[key].motif_enrichment.to_html(open(out_file, 'w'), escape=False, col_space=80)

### V9 - Unclustered

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch /data/leuven/software/biomed/singularity_images/scenicplus/scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()
# Keep only annotated motifs
from pycistarget.utils import *
annot = load_motif_annotations('homo_sapiens',  fname='/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf/snapshots/motifs-v9-nr.hgnc-m0.001-o0.0.tbl')
annot = annot[['Direct_annot', 'Orthology_annot']]
annot = annot.dropna(axis=0, how='all')
motifs = list(set(annot.index))
# cisTarget - All
# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *
for i in range(len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/icistarget/data/make_rankings/v9/CTX_hg38/CTX_hg38_SCREEN3_1kb_bg_with_mask/CTX_hg38_SCREEN3_1kb_bg_with_mask.regions_vs_motifs.rankings.v2.feather'  
    rg = {key: region_sets[key]}
    ctx_db = cisTargetDatabase(db, rg)   
    ctx_db.db_rankings = ctx_db.db_rankings[ctx_db.db_rankings.index.isin(motifs)]
    # Run (r06i01n13)
    cistarget_dict = run_cistarget(ctx_db = ctx_db,
                                   region_sets = rg,
                                   specie = 'homo_sapiens',
                                   auc_threshold = 0.005,
                                   nes_threshold = 3.0,
                                   rank_threshold = 0.05,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf/snapshots/motifs-v9-nr.hgnc-m0.001-o0.0.tbl',
                                   n_cpu = 1,
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v9')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/output_dicts/' + key + '_cisTarget_dict.pkl', 'wb') as f:
      pickle.dump(cistarget_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/html/'+ key + '.html'
    cistarget_dict[key].motif_enrichment.to_html(open(out_file, 'w'), escape=False, col_space=80)

In [3]:
#!/bin/bash

#SBATCH --partition=bigmem
#SBATCH --cluster wice
#SBATCH --account lp_wice_pilot
#SBATCH --mail-type=ALL
#SBATCH --mail-user=carmen.bravogonzalezblas@kuleuven.be
#SBATCH --ntasks=1

#SBATCH --cpus-per-task=20
#SBATCH --time=72:00:00
#SBATCH --mem=750G

#SBATCH --job-name=cisTarget_v9
#SBATCH --output=cisTarget_v9.out
#SBATCH --error=cisTarget_v9.err

singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch,/local_scratch /data/leuven/software/biomed/singularity_images/scenicplus/scenicplus.sif python /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/cisTarget_v9.py



In [4]:
v10 --> 24309 motifs
v9 --> 18431 motifs

24309

# V9 - Unclustered DEM

In [ ]:
# r06i01n13
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch /data/leuven/software/biomed/singularity_images/scenicplus/scenicplus.sif ipython3

In [1]:
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/region_sets.pkl', 'rb')
region_sets = pickle.load(infile)
infile.close()

# Keep only annotated motifs
from pycistarget.utils import *
annot = load_motif_annotations('homo_sapiens',  fname='/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf/snapshots/motifs-v9-nr.hgnc-m0.001-o0.0.tbl')
annot = annot[['Direct_annot', 'Orthology_annot']]
annot = annot.dropna(axis=0, how='all')
motifs = list(set(annot.index))

# Prepare annotation
import pybiomart as pbm
import pandas as pd
import pyranges as pr
biomart_host = 'http://www.ensembl.org'
promoter_space = 500
name = 'hsapiens_gene_ensembl'
dem_log2fc_thr = 0.5,
dem_motif_hit_thr = 3.0,
dem_max_bg_regions = 500,
dataset = pbm.Dataset(name=name,  host=biomart_host)
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot['Chromosome'] = annot['Chromosome'].astype('str')
filterf = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
annot = annot[~filterf]
annot['Chromosome'] = annot['Chromosome'].str.replace(r'(\b\S)', r'chr\1')
annot = annot[annot.Transcript_type == 'protein_coding']
annot = annot.dropna(subset = ['Chromosome', 'Start'])
# Check if chromosomes have chr
check = region_sets[list(region_sets.keys())[0]]
if not any(['chr' in c for c in check[list(check.keys())[0]].df['Chromosome']]):
    annot.Chromosome = annot.Chromosome.str.replace('chr', '')
annot_dem=annot.copy()
# Define promoter space
annot['End'] = annot['Start'].astype(int)+promoter_space
annot['Start'] = annot['Start'].astype(int)-promoter_space
annot = pr.PyRanges(annot[['Chromosome', 'Start', 'End']])

region_set_list = [region_sets[x].df for x in region_sets.keys()]
all_regions = pr.PyRanges(pd.concat(region_set_list))

# DEM
# Load DEM functions
from pycistarget.motif_enrichment_dem import *
for i in range(250,len(region_sets)):
    print(i)
    key = list(region_sets.keys())[i]
    # Create bg
    bg_overlap = all_regions.count_overlaps(region_sets[key])
    bg = bg_overlap[bg_overlap.NumberOverlaps == 0]
    bg = pr.PyRanges(bg.df.sample(frac=0.0005))
    # Preload db, you can also just provide the path to the db. Preloading the database is useful if you want to test different parameters.
    db = '/staging/leuven/stg_00002/lcb/icistarget/data/make_rankings/v9/CTX_hg38/CTX_hg38_SCREEN3_1kb_bg_with_mask/CTX_hg38_SCREEN3_1kb_bg_with_mask.regions_vs_motifs.scores.v2.feather'  
    rg = {key: region_sets[key], 'Background': bg}
    dem_db = DEMDatabase(db, rg)    
    dem_db.db_scores = dem_db.db_scores[dem_db.db_scores.index.isin(motifs)]
    # Run (r06i00n13)
    DEM_dict = DEM(dem_db = dem_db,
                                   region_sets = rg,
                                   contrasts = [[[key],['Background']]],
                                   log2fc_thr = 0.5,
                                   motif_hit_thr = 3.0,
                                   specie = 'homo_sapiens',
                                   max_bg_regions = 5000,
                                   genome_annotation = annot_dem,
                                   promoter_space = promoter_space,
                                   path_to_motif_annotations = '/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf/snapshots/motifs-v9-nr.hgnc-m0.001-o0.0.tbl',
                                   n_cpu = 1,
                                   tmp_dir = '/scratch/leuven/313/vsc31305/ray_spill',
                                   _temp_dir='/scratch/leuven/313/vsc31305/ray_spill',
                                   annotation_version = 'v9')
    # Save pkl
    import pickle
    with open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v9_unclustered/output_dicts/' + key + '_DEM_dict.pkl', 'wb') as f:
      pickle.dump(DEM_dict, f)
    # Save html
    out_file = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v9_unclustered/html/'+ key + '.html'
    DEM_dict.motif_enrichment[key+'_VS_Background'].to_html(open(out_file, 'w'), escape=False, col_space=80)

In [ ]:
#!/bin/bash

#SBATCH --partition=bigmem
#SBATCH --cluster wice
#SBATCH --account lp_wice_pilot
#SBATCH --mail-type=ALL
#SBATCH --mail-user=carmen.bravogonzalezblas@kuleuven.be
#SBATCH --ntasks=1

#SBATCH --cpus-per-task=20
#SBATCH --time=72:00:00
#SBATCH --mem=750G

#SBATCH --job-name=DEM_part5
#SBATCH --output=DEM_part5.out
#SBATCH --error=DEM_part5.err

singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch,/local_scratch /data/leuven/software/biomed/singularity_images/scenicplus/scenicplus.sif python /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v9_unclustered/DEM_part5.py




# Prepare data

## cisTarget v9

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr[list(menr.keys())[0]].motif_enrichment.to_csv(dire+list(menr.keys())[0]+'.tsv', sep='\t')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Region_set'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [3]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v9_unclustered/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Database'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

0
K562_ELK1_ENCSR000EFU
1
GM12878_MEF2C_ENCSR000BNG
2
K562_STAT2_ENCSR000FAT
3
K562_TFDP1_ENCSR017GBO
4
K562_NFIC_ENCSR796ITY
5
K562_E2F7_ENCSR171CAY
6
MCF-7_MAX_ENCSR000BUL
7
MCF-7_CTCF_ENCSR560BUE
8
MCF-7_SREBF1_ENCSR197DJH
9
MCF-7_FOXA1_ENCSR126YEB
10
MCF-7_CTCF_ENCSR000DMS
11
GM12878_E2F4_ENCSR000DYY
12
MCF-7_NFIB_ENCSR582ZOA
13
K562_GATA1_ENCSR000EWM
14
K562_EGR1_ENCSR000BNE
15
K562_NFYB_ENCSR000EGQ
16
HepG2_NFE2L2_ENCSR488EES
17
K562_MEIS2_ENCSR851BNE
18
K562_MAX_ENCSR000BLP
19
HepG2_HNF4A_ENCSR000BLF
20
HepG2_FOXA1_ENCSR000BLE
21
K562_FOXK2_ENCSR508DQA
22
HepG2_MAFK_ENCSR000EDZ
23
K562_BACH1_ENCSR740NPG
24
GM12878_IRF4_ENCSR000BGY
25
GM12878_JUND_ENCSR000DYS
26
GM12878_NR2F1_ENCSR514VYD
27
K562_FOSL1_ENCSR000BMV
28
HepG2_BHLHE40_ENCSR000BID
29
K562_NR2F1_ENCSR970NKQ
30
K562_MNT_ENCSR979QYJ
31
K562_CTCF_ENCSR000DWE
32
IMR-90_FOS_ENCSR124AIG
33
MCF-7_MYC_ENCSR000DMQ
34
MCF-7_GATA3_ENCSR000EWV
35
MCF-7_ESRRA_ENCSR954WVZ
36
HepG2_ATF3_ENCSR000BKE
37
HepG2_ATF2_ENCSR047BUZ
38
GM12878

## cisTarget clustered

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr[list(menr.keys())[0]].motif_enrichment.to_csv(dire+list(menr.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Region_set'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [5]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_clustered/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Database'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

0
K562_ELK1_ENCSR000EFU
1
K562_STAT2_ENCSR000FAT
2
K562_IRF1_ENCSR000EGL
3
K562_TFDP1_ENCSR017GBO
4
MCF-7_JUN_ENCSR176EXN
5
MCF-7_MAX_ENCSR000BUL
6
MCF-7_CTCF_ENCSR560BUE
7
K562_CEBPG_ENCSR490LWA
8
MCF-7_MAFK_ENCSR555PBN
9
GM12878_E2F4_ENCSR000DYY
10
K562_GATA1_ENCSR000EWM
11
K562_NFIC_ENCSR796ITY
12
IMR-90_CEBPB_ENCSR000EFM
13
K562_MEIS2_ENCSR851BNE
14
K562_EGR1_ENCSR024CNP
15
K562_FOS_ENCSR000FAI
16
K562_CTCF_ENCSR000DMA
17
HepG2_MAX_ENCSR000EDS
18
HepG2_HNF4A_ENCSR000BLF
19
K562_FOXK2_ENCSR508DQA
20
K562_ATF4_ENCSR145TSJ
21
GM12878_HSF1_ENCSR009MBP
22
K562_BACH1_ENCSR740NPG
23
GM12878_IRF4_ENCSR000BGY
24
GM12878_JUND_ENCSR000DYS
25
K562_NRF1_ENCSR837EYC
26
K562_NR2F1_ENCSR970NKQ
27
K562_NR2C1_ENCSR742IDN
28
MCF-7_MYC_ENCSR000DMQ
29
GM12878_TCF7_ENCSR501DKS
30
HepG2_ATF2_ENCSR047BUZ
31
PC-3_CTCF_ENCSR359LOD
32
HepG2_IRF3_ENCSR000EEJ
33
MCF-7_GATA3_ENCSR000EWS
34
HepG2_CTCF_ENCSR000DLS
35
K562_JUN_ENCSR000EFS
36
K562_NR2F2_ENCSR000BRS
37
K562_BACH1_ENCSR000EGD
38
K562_MNT_ENCSR979QYJ


## cisTarget unclustered

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr[list(menr.keys())[0]].motif_enrichment.to_csv(dire+list(menr.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Region_set'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [ ]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_unclustered/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Database'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

## cisTarget archetypes

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr[list(menr.keys())[0]].motif_enrichment.to_csv(dire+list(menr.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Region_set'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Region_set'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [ ]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_archetype/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Database'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

## cisTarget public

In [ ]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_public/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/cisTarget_v10_public/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr[key].cistromes['Database'].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr[key].cistromes['Database'][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

## DEM v9 unclustered

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v9_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr.motif_enrichment[list(menr.motif_enrichment.keys())[0]].to_csv(dire+list(menr.motif_enrichment.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v9_unclustered/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v9_unclustered/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Region_set'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Region_set'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [7]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/cistromes_hits_SCREEN/'
#os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Database'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Database'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

0
GM12878_REST_ENCSR000BGF_VS_Background
1
GM12878_ATF2_ENCSR961PPA_VS_Background
2
GM12878_MEF2C_ENCSR000BNG_VS_Background
3
K562_SOX6_ENCSR788RSW_VS_Background
4
K562_NRF1_ENCSR494TDU_VS_Background
5
K562_REST_ENCSR000ATM_VS_Background
6
K562_FOSL1_ENCSR239ZLZ_VS_Background
7
HepG2_FOXA2_ENCSR000BNI_VS_Background
8
HepG2_TCF7L2_ENCSR000EVQ_VS_Background
9
HepG2_CEBPB_ENCSR000EEX_VS_Background
10
GM12878_ETV6_ENCSR626VUC_VS_Background
11
MCF-7_SREBF1_ENCSR197DJH_VS_Background
12
HepG2_FOXA2_ENCSR066EBK_VS_Background
13
K562_MAX_ENCSR000BLP_VS_Background
14
GM12878_JUND_ENCSR000EYV_VS_Background
15
K562_NFYA_ENCSR000EGR_VS_Background
16
MCF-7_GATA3_ENCSR000EWV_VS_Background
17
K562_MGA_ENCSR710WLO_VS_Background
18
HepG2_FOXP1_ENCSR029LBT_VS_Background
19
GM12878_EBF1_ENCSR000BGU_VS_Background
20
MCF-7_RFX5_ENCSR924TVL_VS_Background
21
GM12878_PBX3_ENCSR000BGR_VS_Background
22
MCF-7_ELF1_ENCSR000BSS_VS_Background
23
GM12878_IRF4_ENCSR000BGY_VS_Background
24
K562_RUNX1_ENCSR414TYY_VS_Bac

## DEM unclustered

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr.motif_enrichment[list(menr.motif_enrichment.keys())[0]].to_csv(dire+list(menr.motif_enrichment.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Region_set'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Region_set'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [7]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_unclustered/cistromes_hits_SCREEN/'
#os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Database'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Database'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

0
GM12878_REST_ENCSR000BGF_VS_Background
1
GM12878_ATF2_ENCSR961PPA_VS_Background
2
GM12878_MEF2C_ENCSR000BNG_VS_Background
3
K562_SOX6_ENCSR788RSW_VS_Background
4
K562_NRF1_ENCSR494TDU_VS_Background
5
K562_REST_ENCSR000ATM_VS_Background
6
K562_FOSL1_ENCSR239ZLZ_VS_Background
7
HepG2_FOXA2_ENCSR000BNI_VS_Background
8
HepG2_TCF7L2_ENCSR000EVQ_VS_Background
9
HepG2_CEBPB_ENCSR000EEX_VS_Background
10
GM12878_ETV6_ENCSR626VUC_VS_Background
11
MCF-7_SREBF1_ENCSR197DJH_VS_Background
12
HepG2_FOXA2_ENCSR066EBK_VS_Background
13
K562_MAX_ENCSR000BLP_VS_Background
14
GM12878_JUND_ENCSR000EYV_VS_Background
15
K562_NFYA_ENCSR000EGR_VS_Background
16
MCF-7_GATA3_ENCSR000EWV_VS_Background
17
K562_MGA_ENCSR710WLO_VS_Background
18
HepG2_FOXP1_ENCSR029LBT_VS_Background
19
GM12878_EBF1_ENCSR000BGU_VS_Background
20
MCF-7_RFX5_ENCSR924TVL_VS_Background
21
GM12878_PBX3_ENCSR000BGR_VS_Background
22
MCF-7_ELF1_ENCSR000BSS_VS_Background
23
GM12878_IRF4_ENCSR000BGY_VS_Background
24
K562_RUNX1_ENCSR414TYY_VS_Bac

## DEM archetype

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr.motif_enrichment[list(menr.motif_enrichment.keys())[0]].to_csv(dire+list(menr.motif_enrichment.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Region_set'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Region_set'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [9]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_archetype/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Database'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Database'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

0
MCF-7_GATA3_ENCSR423RTK_VS_Background
1
GM12878_ATF3_ENCSR000BJY_VS_Background
2
K562_ELF4_ENCSR638QHV_VS_Background
3
HepG2_ESRRA_ENCSR000EEW_VS_Background
4
HepG2_CEBPB_ENCSR000EEX_VS_Background
5
K562_SP1_ENCSR991ELG_VS_Background
6
K562_SRF_ENCSR000BLK_VS_Background
7
K562_USF2_ENCSR359NFW_VS_Background
8
K562_BACH1_ENCSR740NPG_VS_Background
9
K562_E2F6_ENCSR000BLI_VS_Background
10
MCF-7_E2F8_ENCSR897ZXU_VS_Background
11
K562_RFX5_ENCSR000EGO_VS_Background
12
K562_STAT1_ENCSR000FAU_VS_Background
13
HepG2_GABPA_ENCSR000BJK_VS_Background
14
K562_CUX1_ENCSR000EFO_VS_Background
15
GM12878_JUNB_ENCSR897MMC_VS_Background
16
K562_MAFK_ENCSR000EGX_VS_Background
17
HepG2_HSF1_ENCSR000EET_VS_Background
18
K562_MAFF_ENCSR000EGI_VS_Background
19
MCF-7_KLF9_ENCSR125ZYC_VS_Background
20
K562_CEBPG_ENCSR490LWA_VS_Background
21
HepG2_MAFK_ENCSR000EEB_VS_Background
22
GM12878_BHLHE40_ENCSR987MTA_VS_Background
23
GM12878_ATF2_ENCSR961PPA_VS_Background
24
HepG2_MYBL2_ENCSR000BRO_VS_Background
25
K5

## DEM clustered

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr.motif_enrichment[list(menr.motif_enrichment.keys())[0]].to_csv(dire+list(menr.motif_enrichment.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Region_set'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Region_set'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [ ]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_clustered/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Database'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Database'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [2]:
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_public/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = path+'tsv/'

for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    menr.motif_enrichment[list(menr.motif_enrichment.keys())[0]].to_csv(dire+list(menr.motif_enrichment.keys())[0]+'.tsv', sep='\t')

In [ ]:
import pandas as pd
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_public/'
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_public/cistromes_hits/'
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Region_set'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Region_set'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Region_set'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)

In [ ]:
import pandas as pd
import os
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_public/'
from os import listdir
from os.path import isfile, join
dicts = listdir(path+'output_dicts')
dire = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/ChIP-seq/motif_enrichment_benchmark/DEM_v10_public/cistromes_hits_SCREEN/'
os.mkdir(dire)
for i in range(len(dicts)):
    print(i)
    d = dicts[i]
    f = path+'output_dicts/'+d
    import pickle
    infile = open(f, 'rb')
    menr = pickle.load(infile)
    infile.close()
    for key in menr.cistromes['Database'].keys():
        print(key)
        tf = key.split('_')[1]
        dict_keys = menr.cistromes['Database'][key].keys()
        name = [name for name in dict_keys if tf+'_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)
        name = [name for name in dict_keys if tf+'_extended_(' in name]
        if len(name) == 1:
            save_name = name[0].split('_(')[0]
            pd.DataFrame(menr.cistromes['Database'][key][name[0]]).to_csv(dire+key+'_'+save_name+'.tsv', sep='\t', header=False, index=False)